<a href="https://colab.research.google.com/github/pablojrios/fluence_maps/blob/master/tf2_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compute predictions on a TF dataset using an stored .h5 model

In [1]:
#%cd /content/drive/My\ Drive/Healthcare/Radioterapia/data/ciolaplata
#!unrar x /content/drive/My\ Drive/Healthcare/Radioterapia/Mapas\ CIO\ La\ Plata/Mapas\ Calculados/2019.rar
#!ls -l 2015/*dcm | wc -l

import os

%cd -q '/content'
if os.path.exists('fluence_maps'):
  !rm -fr fluence_maps

GIT_USERNAME = "pablojrios"
GIT_TOKEN = "1d88a0b85d2b00a03796e4d8b7e5f7b249b12f9b"
!git clone -s https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/fluence_maps.git

Cloning into 'fluence_maps'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (212/212), done.
remote: Total 252 (delta 144), reused 95 (delta 39), pack-reused 0
Receiving objects: 100% (252/252), 1.55 MiB | 21.73 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [2]:
import numpy as np
import tensorflow as tf
%cd '/content/fluence_maps'
from util.dataset import _tfrecord_dataset_type_from_folder, _parse_jpeg_image_function
from util.preprocess import rescale_0_to_1
import os
import pandas as pd

/content/fluence_maps


In [3]:
print('Tensorflow version = {}'.format(tf.__version__))
print('Executing eagerly = {}'.format(tf.executing_eagerly()))

Tensorflow version = 2.4.0
Executing eagerly = True


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls -l '/content/drive/My Drive/Healthcare/Radioterapia/data/ciolaplata/models'

total 845867
-rw------- 1 root root 132221552 Jan 16 19:45 1610826316.h5
-rw------- 1 root root 156500040 Jan 16 20:13 1610828018.h5
-rw------- 1 root root 156500040 Jan 16 20:53 1610830410.h5
-rw------- 1 root root 132221552 Jan 16 21:10 1610831434.h5
-rw------- 1 root root 132221552 Jan 17 02:08 1610849307.h5
-rw------- 1 root root 156500040 Jan 17 18:09 1610906968.h5


In [6]:
#============================DEFINE YOUR ARGUMENTS==============================
# base data directory
ARG_DATASET_DIR='/content/drive/My Drive/Healthcare/Radioterapia/data/ciolaplata'
# folder under ARG_DATASET_DIR path.
ARG_TFDATASET_FOLDER='tfds.2019.localnorm.ovs95x8.0'
ARG_MODEL_NAME = '1610906968'
# 'train', 'validation', 'test' 
ARG_PART = 'train'
ARG_TRANSFORM_GAMMA=False

In [7]:
tfdataset_dir = os.path.join(ARG_DATASET_DIR, ARG_TFDATASET_FOLDER)
raw_test = _tfrecord_dataset_type_from_folder(tfdataset_dir, ARG_PART)
print(raw_test)

<TFRecordDatasetV2 shapes: (), types: tf.string>


In [8]:
# Apply this function to each item in the dataset using the map method:
num_workers = 8
IMG_SIZE = 256
normalization_fn = rescale_0_to_1
test = raw_test.map(lambda e: _parse_jpeg_image_function(e, IMG_SIZE, normalization_fn, transform_gamma=ARG_TRANSFORM_GAMMA), num_parallel_calls=num_workers)
print(test)

<ParallelMapDataset shapes: ((256, 256, 3), (), ()), types: (tf.float32, tf.float32, tf.string)>


In [9]:
gamma_values = test.map(lambda image, gamma, filename: gamma)
gamma_values = np.array(list(gamma_values.as_numpy_iterator()))
BATCH_SIZE = 32 # mae puede variar según batch size.
test_batches = test.batch(BATCH_SIZE)
print(test_batches)

<BatchDataset shapes: ((None, 256, 256, 3), (None,), (None,)), types: (tf.float32, tf.float32, tf.string)>


In [10]:
gamma_values

array([96.3577 , 99.33116, 99.80857, ..., 93.3324 , 94.653  , 93.0731 ],
      dtype=float32)

In [11]:
# load model
dir = os.path.join(ARG_DATASET_DIR, "models")
saved_model_dir = '{}/{}.h5'.format(dir, ARG_MODEL_NAME)
print(f'Loading model {saved_model_dir}...')
loaded_model = tf.keras.models.load_model(saved_model_dir)

Loading model /content/drive/My Drive/Healthcare/Radioterapia/data/ciolaplata/models/1610906968.h5...


In [12]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 8, 8, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 14,977,857
Trainable params: 12,062,209
Non-trainable params: 2,915,648
_________________________________________________________________


In [13]:
# Evaluate dataset with the loaded model to calculate loss (mae) because
# metric value could differ from the one reported during training.
tmp_test_batches = test_batches.map(lambda image, gamma, filename: (image, gamma))
print(tmp_test_batches)
loss, mse = loaded_model.evaluate(tmp_test_batches, workers=num_workers, verbose=0)
print('\n\nLoaded model, test loss: {:5.4f}'.format(loss))
print('Loaded model, test mse: {:5.4f}'.format(mse))

<MapDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.float32)>


Loaded model, test loss: 0.4885
Loaded model, test mse: 0.5178


In [14]:
# For VGG16 only
#inputs=loaded_model.get_layer("vgg16").input
#outputs=loaded_model.get_layer("vgg16").output
#cnn_part=tf.keras.models.Model(inputs, outputs)
#cnn_part.summary()
#predictions = cnn_part.predict(tmp_test_batches)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [15]:
# For VGG16 only
#predictions[0][:,:,3]

array([[0.31196553, 0.36073306, 0.36109102, 0.3347852 , 0.3657075 ,
        0.37897533, 0.37125105, 0.34027943],
       [0.2777101 , 0.33441287, 0.24188238, 0.01735653, 0.23871636,
        0.35668796, 0.3561841 , 0.31457597],
       [0.2777711 , 0.3210093 , 0.08035876, 0.        , 0.15479   ,
        0.34747148, 0.35389912, 0.30943745],
       [0.27800325, 0.2998556 , 0.        , 0.        , 0.09436014,
        0.35097644, 0.35463405, 0.30961794],
       [0.27790347, 0.29005685, 0.        , 0.        , 0.12613538,
        0.35082242, 0.3541686 , 0.30961472],
       [0.27813825, 0.31236264, 0.06818202, 0.        , 0.23630056,
        0.35277843, 0.35292882, 0.30970085],
       [0.2558931 , 0.29350126, 0.22169833, 0.13289385, 0.26399443,
        0.31686944, 0.3085425 , 0.28057548],
       [0.24316308, 0.24193086, 0.24892177, 0.25191736, 0.25363803,
        0.24673128, 0.23854925, 0.23250261]], dtype=float32)

In [16]:
# For VGG16 only
#global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
#feature_batch_average = global_average_layer(predictions)
#feature_batch_average[0]

<tf.Tensor: shape=(512,), dtype=float32, numpy=
array([9.91073158e-03, 0.00000000e+00, 3.79420757e-01, 2.52264321e-01,
       1.23403661e-01, 0.00000000e+00, 6.20641687e-04, 0.00000000e+00,
       6.78711757e-03, 1.28019564e-02, 0.00000000e+00, 0.00000000e+00,
       2.36113995e-01, 0.00000000e+00, 4.03580628e-03, 2.88030148e-01,
       0.00000000e+00, 3.44557166e-01, 2.02771407e-02, 0.00000000e+00,
       7.31099490e-03, 0.00000000e+00, 7.64280558e-03, 1.96990948e-02,
       1.15046911e-02, 0.00000000e+00, 1.60866648e-01, 0.00000000e+00,
       2.06571773e-01, 0.00000000e+00, 1.25366852e-01, 2.40212288e-02,
       1.73449114e-01, 1.28949910e-01, 2.90632457e-01, 0.00000000e+00,
       4.83654171e-01, 2.78702209e-04, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 4.60051119e-01, 0.00000000e+00,
       0.00000000e+00, 1.59314185e-01, 1.79431774e-03, 1.68199781e-02,
       0.00000000e+00, 2.25453839e-01, 1.27722248e-02, 0.00000000e+00,
       9.06848535e-03, 0.0000

In [17]:
# For VGG16 only
#from math import isclose
#a=predictions[0][:,:,3].mean()
#b=feature_batch_average[0][3].numpy()
#isclose(a, b, rel_tol=1e-7)

False

In [18]:
# Make predictions
predictions = loaded_model.predict(tmp_test_batches)
# from (1121,1) to (1121,); ie.: ndim = 2 to ndim = 1
predictions = predictions.reshape(-1)

In [19]:
tmp_test_batches = test.map(lambda image, gamma, filename: (filename, gamma))
lst = [(filename.numpy().decode('utf-8'), gamma.numpy()) for filename, gamma in tmp_test_batches]
lst2 = [(e[0], e[1], p) for e, p in zip(lst, predictions)]

# armar un pandas dataframe con el test set completo
df = pd.DataFrame(lst2, columns=['filename', 'actual gamma', 'predicted gamma'])
dir = os.path.join(ARG_DATASET_DIR, "predictions")
predictions_file_path = '{}/predicted_gamma_{}.{}.csv'.format(dir, ARG_MODEL_NAME, ARG_PART)
df.to_csv(predictions_file_path, index=False)
print(f'Predictions saved to {predictions_file_path}.')

Predictions saved to /content/drive/My Drive/Healthcare/Radioterapia/data/ciolaplata/predictions/predicted_gamma_1610906968.train.csv.


In [20]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.
